In [2]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 1.32 s, sys: 265 ms, total: 1.59 s
Wall time: 2.52 s


In [4]:
test_data = pd.read_csv("test_data/test_data.csv", header=None)
test_data

,0,1,2,3,4
0,0.241110,5,0.0,9.24,-1.0
1,0.191735,5,1.0,9.31,-1.0
2,0.155147,5,2.0,9.72,-1.0
3,0.135726,5,3.0,9.18,-1.0
4,0.126615,5,4.0,6.73,-1.0
...,...,...,...,...,...
644,0.347004,3,20.0,5.94,-1.0
645,0.334706,3,21.0,5.03,-1.0
646,0.299760,3,22.0,4.10,-1.0
647,0.252274,3,23.0,3.93,-1.0


In [5]:
batch_data = pd.read_csv("test_data/batch_data.csv", header=None)
batch_data

,0,1,2,3
0,5,0.0,9.24,-1.0
1,5,1.0,9.31,-1.0
2,5,2.0,9.72,-1.0
3,5,3.0,9.18,-1.0
4,5,4.0,6.73,-1.0
...,...,...,...,...
644,3,20.0,5.94,-1.0
645,3,21.0,5.03,-1.0
646,3,22.0,4.10,-1.0
647,3,23.0,3.93,-1.0


In [6]:
test_data200 = test_data.head(200)
test_data200

,0,1,2,3,4
0,0.241110,5,0.0,9.24,-1.0
1,0.191735,5,1.0,9.31,-1.0
2,0.155147,5,2.0,9.72,-1.0
3,0.135726,5,3.0,9.18,-1.0
4,0.126615,5,4.0,6.73,-1.0
...,...,...,...,...,...
195,0.133863,6,3.0,7.37,-1.0
196,0.124057,6,4.0,6.94,-1.0
197,0.122383,6,5.0,6.69,-1.0
198,0.133539,6,6.0,7.07,-1.0


In [7]:
batch_data200 = batch_data.head(200)
batch_data200

,0,1,2,3
0,5,0.0,9.24,-1.0
1,5,1.0,9.31,-1.0
2,5,2.0,9.72,-1.0
3,5,3.0,9.18,-1.0
4,5,4.0,6.73,-1.0
...,...,...,...,...
195,6,3.0,7.37,-1.0
196,6,4.0,6.94,-1.0
197,6,5.0,6.69,-1.0
198,6,6.0,7.07,-1.0


In [8]:
batch_data200.to_csv('test_data/batch_data200.csv', index=False)
test_data200.to_csv('test_data/test_data200.csv', index=False)